In [ ]:
import geopandas as gpd
from PIL.Image import preinit
from shapely import delaunay_triangles
from shapely.geometry.multilinestring import MultiLineString

polywholes = gpd.read_file('poly_holes.geojson').to_crs(32636)
# polywholes.explore()

poly_w_holes = polywholes.geometry.iloc[0]

# def poly_with_holes_to_multipoly(polygon):

# m1 = polywholes.explore(color='red')
# # gpd.GeoDataFrame(geometry=[delaunay_triangles(poly_w_holes)],crs=polywholes.crs).explode().explore(m=m1)
# m1

In [ ]:
from shapely import LineString
from shapely.geometry import Polygon
from scipy.spatial.distance import cdist
import numpy as np
from shapely.ops import unary_union
from shapely.ops import nearest_points

inner_geoms = [Polygon(ring) for ring in poly_w_holes.interiors]
inner_centroids = [list(poly.representative_point().coords)[0] for poly in inner_geoms]
outside_poly = Polygon(poly_w_holes.exterior)
distances = cdist(inner_centroids, inner_centroids)
distances[distances == 0] = np.inf
n = len(inner_centroids)
lines = []
for i in range(len(inner_centroids) - 1):
    col, row = np.unravel_index(np.argmin(distances), (n, n))
    distances[col, row] = np.inf
    distances[row, col] = np.inf
    lines.append(LineString([inner_centroids[col], inner_centroids[row]]))
row_counts = np.sum(distances != np.inf, axis=1)
target_row_index = np.where(row_counts == n - 2)[0]
lines += [LineString([inner_centroids[last_ind], nearest_points(inner_geoms[last_ind], outside_poly.exterior)[1]]) for
          last_ind in target_row_index]
print(lines)

In [ ]:
from app.gen_planner.python.src.utils import geometry_to_multilinestring
from shapely import LineString
from shapely.ops import nearest_points
from shapely.ops import unary_union, polygonize

buffered_poly_w_holes = poly_w_holes.buffer(0.1)
lines = [LineString(nearest_points(poly, buffered_poly_w_holes.exterior)).buffer(0.1, resolution=1).exterior for poly in
         buffered_poly_w_holes.interiors]
polygons = list(polygonize(unary_union([geometry_to_multilinestring(poly_w_holes)] + lines)))
repr = poly_w_holes.representative_point()
for poly in polygons:
    if poly.contains(repr):
        result = poly
        break
result

In [ ]:
from app.gen_planner.python.src import GenPlanner, func_zones, terr_zones
from shapely import Point

generated_zones, generated_roads = GenPlanner(gpd.GeoDataFrame(geometry=[result], crs=32636),
                                              dev_mod=True).features2terr_zones(
    funczone=func_zones.residential_func_zone)
generated_zones.explore(
    tiles='Cartodb Positron', column='territory_zone', cmap='Dark2')

In [1]:
from app.gen_planner.python.src import territory_splitter
from app.gen_planner.python.src.utils import geometry_to_multilinestring, explode_linestring
import geopandas as gpd
import pandas as pd

zone = gpd.read_file('../scenarios_data/project Шлиссельбург.geojson')
zone_roads = gpd.read_file('../scenarios_data/roads_shlis.geojson')

# zone_roads.geometry = zone_roads.to_crs(32636).geometry.buffer(5)
#
# zone_splitted = territory_splitter(zone, zone_roads)
# splitters_lines  = gpd.GeoDataFrame(geometry=pd.Series(zone_splitted.geometry.apply(geometry_to_multilinestring).explode().apply(explode_linestring)).explode(ignore_index=True), crs=zone_splitted.crs)
# splitters_lines  = gpd.GeoDataFrame(geometry=zone_splitted.geometry.apply(geometry_to_multilinestring).explode().to_list(), crs=zone_splitted.crs)
# splitters_lines.geometry = splitters_lines.geometry.centroid.buffer(1,resolution=1)
# zone_roads.sjoin(splitters_lines,how="inner",predicate="intersects").explore()

Available workers count 16


In [2]:
from shapely import line_merge, MultiLineString

splitted_lines = gpd.GeoSeries(
    pd.Series(zone_roads.to_crs(32636).geometry.apply(geometry_to_multilinestring).explode().apply(
        explode_linestring)).explode(
        ignore_index=True), crs=32636).normalize().set_precision(0.00000001).unique()

# splitted_lines = zone_splitted.to_crs(32636).geometry.apply(geometry_to_multilinestring).explode().to_list()

lines = line_merge(MultiLineString(splitted_lines))
#
# gpd.GeoDataFrame(geometry=lines,crs=32636).explode(ignore_index=True).reset_index().explore(column='index')

print(len(lines.geoms))
print(len(splitted_lines))

241
1006


In [6]:
from app.gen_planner.python.src import GenPlanner, func_zones, terr_zones
from shapely import Point

zones_ratio_dict = {'right_lower': 0.225, 'right_upper': 0.225, 'left_upper': 0.225, 'left_lower': 0.225,
                    'middle_lower': 0.1, 'middle_upper': 0.1, 'free_zone1': 0.1, 'free_zone2': 0.1}
fixed_zones = gpd.GeoDataFrame()
# middle_lower = Point(31.015, 59.911)
middle_upper = Point(31.022, 59.922)
right_upper = Point(31.046, 59.922)
right_lower = Point(31.043, 59.911)
left_upper = Point(30.999, 59.927)
left_lower = Point(30.993, 59.912)

fixed_2zones = gpd.GeoDataFrame(
    {'fixed_zone': [terr_zones.transport_terr, terr_zones.recreation_terr]},
    geometry=[left_lower, middle_upper],
    crs=4326)

fixed_4zones = gpd.GeoDataFrame(
    {'fixed_zone': [terr_zones.transport_terr, terr_zones.recreation_terr, terr_zones.special_terr,
                    terr_zones.business_terr]},
    geometry=[right_upper, right_lower, left_upper, left_lower],
    crs=4326)

fixed_zones = fixed_4zones

territory = zone.copy()
# territory = zone_splitted.copy()

generated_zones, generated_roads = GenPlanner(zone, zone_roads,dev_mod = True).features2terr_zones2blocks(
    funczone=func_zones.residential_func_zone, fixed_terr_zones=fixed_zones)

2025-06-05 15:52:27.453 | INFO     | app.gen_planner.python.src.genplanner.genplanner:__init__:50 - Dev mod activated, no more ProcessPool


Статус: Optimal


In [7]:
import pandas as pd

m1 = pd.concat([generated_zones,
                fixed_zones.rename(columns={'fixed_zone': 'territory_zone'}).to_crs(generated_zones.crs)]).explore(
    tiles='Cartodb Positron', column='territory_zone', cmap='Dark2')
m1

In [ ]:
generated_roads.explore(column='road_lvl')